In [2]:
#Load the dataset
import pandas as pd
df = pd.read_csv('insurance.csv')
df.head()

,age,affordability,have_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
#split the dataset into training and testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df[['age', 'affordability']], df.have_insurance, train_size = 0.7)
x_train.shape

(19, 2)

In [75]:
#scale the 'age' feature values between 0 and 1 cos we have affordability values in range between 0 and 1
x_train_scaled = x_train.copy()
x_train_scaled['age'] = x_train_scaled['age'] / 100

x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age'] / 100

x_train_scaled['age'].head(), x_train_scaled.shape

(0     0.22
 18    0.19
 26    0.23
 20    0.21
 16    0.25
 Name: age, dtype: float64,
 (19, 2))

In [76]:
#Build and train the neural network
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(1, input_shape = (2,), activation="sigmoid", 
                        kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(
    optimizer="adam",
    metrics=['accuracy'],
    loss="binary_crossentropy"
)

model.fit(x_train_scaled, y_train, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 0.7505 - accuracy: 0.4737
Epoch 2/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.7501 - accuracy: 0.4737
Epoch 3/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.7496 - accuracy: 0.4737
Epoch 4/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.7492 - accuracy: 0.4737
Epoch 5/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.7488 - accuracy: 0.4737
Epoch 6/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7484 - accuracy: 0.4737
Epoch 7/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.7479 - accuracy: 0.4737
Epoch 8/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.7475 - accuracy: 0.4737
Epoch 9/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.7471 - accuracy: 0.4737
Epoch 10/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.7466 - accuracy: 0.4737
Epoch 11/1000
1/

In [77]:
#evaluate the model on test data
model.evaluate(x_test_scaled, y_test)

1/1 [==============================] - 0s 352ms/step - loss: 0.5647 - accuracy: 0.7778


[0.5646626949310303, 0.7777777910232544]

In [80]:
#coefficients and bias for the model
coef, intercept = model.get_weights()
coef, intercept

(array([[1.1798273],
        [0.547167 ]], dtype=float32),
 array([-0.66353875], dtype=float32))

In [81]:
#check the result for age(22) and affordability(1) with our own 
# 'activation' and 'prediction' function
#original result = 0
#predicted result = 0
#test passed
import math
def sigmoid(x):
    res = 1 / (1 + math.exp(-x))
    print(res)

def predictionFunction(age, affordability):
    weighted_sum =  age * coef[0] + affordability * coef[1] + intercept
    return weighted_sum

age = x_train_scaled['age'][0]
affordability = x_train_scaled['affordability'][0]

sigmoid(predictionFunction(age, affordability))

0.5357365268510579


In [82]:
#Test our model with test dataset
model.predict(x_test_scaled)

array([[0.6300738 ],
       [0.6464162 ],
       [0.42033732],
       [0.49634165],
       [0.6382849 ],
       [0.52398086],
       [0.6328194 ],
       [0.5532918 ],
       [0.6078174 ]], dtype=float32)

<h2>Now Try to implement the gradient descent algorithm to find coefficent and intercept calculated by keras internally</h2>

<h3>Numpy sigmoid function</h3>

In [83]:
#sigmoid activation function
#input: set of integer or float values 
#output: set of values between 0 and 1
import numpy as np
def sigmoid_np(x):
    return 1 / (1 + np.exp(-x))

<h3>Log loss function</h3>

In [84]:
def log_loss(y_true, y_pred):
    epsilon = 1e-15
    y_pred_new = [max(i, epsilon) for i in y_pred]
    y_pred_new = [min(i, (1-epsilon)) for i in y_pred_new]
    y_pred_new = np.array(y_pred_new)
    return -(np.mean(y_true * np.log(y_pred_new) + (1 - y_true) * np.log(1 - y_pred_new)))

<h3>Gradient descent algorithm to find 'w1', 'w2' and 'b'</h3>

In [91]:
def gradient_descent_nn(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 =1 
    b = 0
    n = len(age)
    learning_rate = 0.001
    for i in  range(epochs):
        #calculate the cost (or) loss
        weighted_sum = w1 * age + w2 * affordability + b
        y_predicted = sigmoid_np(weighted_sum)
        cost = log_loss(y_true, y_predicted)

        #calculate the derivaties of w1, w2, b for slope(direction)
        w1d = 1/n * np.dot(np.transpose(age),(y_predicted - y_true))
        w2d = 1/n * np.dot(np.transpose(affordability), (y_predicted - y_true))
        bd = np.mean(y_predicted-y_true)

        #calculate the next w1, w2, b
        w1 = w1 - learning_rate * w1d
        w2 = w2 - learning_rate * w2d
        b = b - learning_rate * bd

        print(f'epoch: {i}, w1: {w1}, w2: { w2}, b: {b}, loss: { cost }')
        if(cost <= loss_thresold):
            break

    return w1, w2, b

In [92]:
gradient_descent_nn(x_train_scaled['age'], x_train_scaled['affordability']
,y_train, 1000, 0.4631)

epoch: 0, w1: 0.9999505949782878, w2: 0.9998627216235565, b: -0.00024655437061309486, loss: 0.7505101238434837
epoch: 1, w1: 0.9999012134518652, w2: 0.999725483720793, b: -0.0004930448311583201, loss: 0.7504280598137811
epoch: 2, w1: 0.9998518554184715, w2: 0.9995882862900796, b: -0.0007394713886300576, loss: 0.750346040722143
epoch: 3, w1: 0.9998025208758426, w2: 0.9994511293297798, b: -0.0009858340500305987, loss: 0.7502640665519018
epoch: 4, w1: 0.9997532098217112, w2: 0.9993140128382497, b: -0.00123213282237014, loss: 0.7501821372863873
epoch: 5, w1: 0.9997039222538071, w2: 0.9991769368138387, b: -0.0014783677126667776, loss: 0.7501002529089265
epoch: 6, w1: 0.9996546581698568, w2: 0.999039901254889, b: -0.001724538727946502, loss: 0.7500184134028437
epoch: 7, w1: 0.9996054175675837, w2: 0.9989029061597356, b: -0.001970645875243193, loss: 0.7499366187514602
epoch: 8, w1: 0.9995562004447084, w2: 0.9987659515267071, b: -0.0022166891615986147, loss: 0.7498548689380954
epoch: 9, w1: 0.

(0.9618426688003426, 0.8824052884160448, -0.21607564357023662)

In [88]:
coef, intercept

(array([[1.1798273],
        [0.547167 ]], dtype=float32),
 array([-0.66353875], dtype=float32))

In [66]:
x_train_scaled[['age']], x_train_scaled['age']

(     age
 0   0.22
 18  0.19
 26  0.23
 20  0.21
 16  0.25
 7   0.60
 24  0.50
 21  0.26
 19  0.18
 3   0.52
 22  0.40
 12  0.27
 14  0.49
 27  0.46
 1   0.25
 4   0.46
 23  0.45
 25  0.54
 8   0.62,
 0     0.22
 18    0.19
 26    0.23
 20    0.21
 16    0.25
 7     0.60
 24    0.50
 21    0.26
 19    0.18
 3     0.52
 22    0.40
 12    0.27
 14    0.49
 27    0.46
 1     0.25
 4     0.46
 23    0.45
 25    0.54
 8     0.62
 Name: age, dtype: float64)